In [3]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from datetime import date
import pandas as pd
import numpy as np 
import plotly.figure_factory as ff

from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import random 
import warnings
import operator
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [2]:
#need to figure out a way to manage missing values

## Exploration of the data

In [161]:

train = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/train.csv", nrows = None)
test = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/test.csv")


#how to set wdir ??
print ("Train Dataset: Rows, Columns: ", train.shape)
print ("Test Dataset: Rows, Columns: ", test.shape)
#weird that we have bigger test set than train set

Train Dataset: Rows, Columns:  (9557, 143)
Test Dataset: Rows, Columns:  (23856, 142)


## Feature engineering

Code written by me

In [156]:
def feature_creation_function(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('bed_density', 'bedrooms', 'rooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('person_per_room', 'r4t3', 'rooms'),
                 ('person_per_room', 'r4t3', 'bedrooms'),
                 ('rez_esc_escolari', 'rez_esc', 'escolari'),
                 ('rez_esc_r4t1', 'rez_esc', 'r4t1'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                 ('male_over_female', 'r4h3', 'r4m3'),
                 ('man12plus_over_women12plus', 'r4h2', 'r4m2'),
                 ('pesioner_over_working', 'hogar_mayor', 'hogar_adul'),
                 ('children_over_working', 'hogar_nin', 'hogar_adul'),
                 ('education_fraction', 'escolari', 'age')
                 #('', '', ''),
                ]
    
    
  
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('non_bedrooms', 'rooms', 'bedrooms'),
                 ('people_not_linving_', 'tamhog', 'r4t3')
                 ]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
    
    

In [157]:
#uses do_features

def preprocess(df): #reencode idhogar
    # encode the idhogar - houshold level identifier
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])
    # create aggregate features
    

  

### data preprocessing

In [158]:
#mapping yes/no to float values
def yes_no_issue_solver(df):
    df['dependency'] = df['dependency'].replace({"yes": 1, "no": 0}).astype(np.float64)
    df['edjefe'] = df['edjefe'].replace({"yes": 1, "no": 0}).astype(np.float64)
    df['edjefa'] = df['edjefa'].replace({"yes": 1, "no": 0}).astype(np.float64)
    


In [159]:
def recoding_ordinal(df):  #create ordinal variable instead of OHE   
    df['walls'] = np.argmax(np.array(df[['epared1', 'epared2', 'epared3']]), axis = 1) #Get quality indexes
    df['roof'] = np.argmax(np.array(df[['etecho1', 'etecho2', 'etecho3']]), axis = 1)
    df['floor'] = np.argmax(np.array(df[['eviv1', 'eviv2', 'eviv3']]), axis = 1)
    
    power =[]
    for i, row in df.iterrows():
        if row['noelec'] == 1:
            power.append(0)
        elif row['coopele'] == 1:
            power.append(1)
        elif row['public'] == 1:
            power.append(2)
        elif row['planpri'] == 1:
            power.append(3)
        else:
            power.append(np.nan)
            
    df['power'] = power
    df['power-missing'] = df['power'].isnull()
          
            
#column to drop : 'noelec', 'coopele', 'public', 'planpri', 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3'            
# data = data.drop(columns = ['etecho1', 'etecho2', 'etecho3'])

In [105]:
## Feature engineering - Aproach 1 - Handmade features

In [118]:
#list(train)

In [162]:
def agg_function_household(df): #create a lot of features
    
    list_aggr_mean = ['rez_esc', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 'parentesco2',
             'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12',
             'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',]

    list_other = ['escolari', 'age', 'fe_education_fraction']

    for item in list_aggr_mean:
        group_mean = df[item].groupby(df['idhogar']).mean()
        new_col = item + '_aggr_MEAN'
        df[new_col] = group_mean


    for item in list_other:
        for function in ['mean','std','min','max','sum']:
            group= df[item].groupby(df['idhogar']).agg(function)
            new_col = item + '_' + function.upper()
            df[new_col] = group


In [151]:
def clean_column(df):
    df.drop(['idhogar','Id', 'tamhog', 'agesq', 'hogar_adul', 'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'hhsize', 'female', 'area2'],axis=1, inplace=True)
    
    
    

In [163]:
for df_ in [train, test]:
    feature_creation_function(df_)
    preprocess(df_) #reencode idhogar
    yes_no_issue_solver(df_)
    recoding_ordinal(df_)
    agg_function_household(df_)
    clean_column(df_)

In [166]:
list(train)

['v2a1',
 'hacdor',
 'rooms',
 'hacapo',
 'v14a',
 'refrig',
 'v18q',
 'v18q1',
 'r4h1',
 'r4h2',
 'r4h3',
 'r4m1',
 'r4m2',
 'r4m3',
 'r4t1',
 'r4t2',
 'r4t3',
 'tamviv',
 'escolari',
 'rez_esc',
 'paredblolad',
 'paredzocalo',
 'paredpreb',
 'pareddes',
 'paredmad',
 'paredzinc',
 'paredfibras',
 'paredother',
 'pisomoscer',
 'pisocemento',
 'pisoother',
 'pisonatur',
 'pisonotiene',
 'pisomadera',
 'techozinc',
 'techoentrepiso',
 'techocane',
 'techootro',
 'cielorazo',
 'abastaguadentro',
 'abastaguafuera',
 'abastaguano',
 'public',
 'planpri',
 'noelec',
 'coopele',
 'sanitario1',
 'sanitario2',
 'sanitario3',
 'sanitario5',
 'sanitario6',
 'energcocinar1',
 'energcocinar2',
 'energcocinar3',
 'energcocinar4',
 'elimbasu1',
 'elimbasu2',
 'elimbasu3',
 'elimbasu4',
 'elimbasu5',
 'elimbasu6',
 'epared1',
 'epared2',
 'epared3',
 'etecho1',
 'etecho2',
 'etecho3',
 'eviv1',
 'eviv2',
 'eviv3',
 'dis',
 'male',
 'estadocivil1',
 'estadocivil2',
 'estadocivil3',
 'estadocivil4',
 '

In [168]:
train = train.query('parentesco1==1')

In [169]:
train.corr()['Target'].map(lambda x: abs(x)).sort_values(ascending = False)

Target                       1.000000
escolari                     0.333791
meaneduc                     0.333652
fe_children_over_working     0.320817
floor                        0.307605
walls                        0.298737
cielorazo                    0.295249
eviv3                        0.293540
epared3                      0.280947
roof                         0.278540
pisomoscer                   0.275452
fe_rent_per_room             0.266982
hogar_nin                    0.266309
etecho3                      0.261142
r4t1                         0.260917
v2a1                         0.259938
paredblolad                  0.254469
fe_children_fraction         0.248346
fe_rent_per_person           0.245791
edjefe                       0.235687
instlevel8                   0.235102
fe_human_bed_density         0.234954
overcrowding                 0.234954
fe_education_fraction        0.232205
v18q                         0.230644
fe_person_per_room           0.227632
fe_mobile_de

In [107]:
#train.drop([train for train in train.columns if train.startswith('SQB') or train == 'agesq'], axis = 1)

In [52]:
list(preprocess(train))

['Id',
 'v2a1',
 'hacdor',
 'rooms',
 'hacapo',
 'v14a',
 'refrig',
 'v18q',
 'v18q1',
 'r4h1',
 'r4h2',
 'r4h3',
 'r4m1',
 'r4m2',
 'r4m3',
 'r4t1',
 'r4t2',
 'r4t3',
 'tamhog',
 'tamviv',
 'escolari',
 'rez_esc',
 'hhsize',
 'paredblolad',
 'paredzocalo',
 'paredpreb',
 'pareddes',
 'paredmad',
 'paredzinc',
 'paredfibras',
 'paredother',
 'pisomoscer',
 'pisocemento',
 'pisoother',
 'pisonatur',
 'pisonotiene',
 'pisomadera',
 'techozinc',
 'techoentrepiso',
 'techocane',
 'techootro',
 'cielorazo',
 'abastaguadentro',
 'abastaguafuera',
 'abastaguano',
 'public',
 'planpri',
 'noelec',
 'coopele',
 'sanitario1',
 'sanitario2',
 'sanitario3',
 'sanitario5',
 'sanitario6',
 'energcocinar1',
 'energcocinar2',
 'energcocinar3',
 'energcocinar4',
 'elimbasu1',
 'elimbasu2',
 'elimbasu3',
 'elimbasu4',
 'elimbasu5',
 'elimbasu6',
 'epared1',
 'epared2',
 'epared3',
 'etecho1',
 'etecho2',
 'etecho3',
 'eviv1',
 'eviv2',
 'eviv3',
 'dis',
 'male',
 'female',
 'estadocivil1',
 'estadocivil

Lets go step by step to have decent features 

In [ ]:
# plot feature importance for sklearn decision trees    
def feature_importance(forest, X_train, display_results=True):
    ranked_list = []
    zero_features = []
    
    importances = forest.feature_importances_

    indices = np.argsort(importances)[::-1]
    
    if display_results:
        # Print the feature ranking
        print("Feature ranking:")

    for f in range(X_train.shape[1]):
        if display_results:
            print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]) + " - " + X_train.columns[indices[f]])
        
        ranked_list.append(X_train.columns[indices[f]])
        
        if importances[indices[f]] == 0.0:
            zero_features.append(X_train.columns[indices[f]])
            
    return ranked_list, zero_features

In [32]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('bed_density', 'bedrooms', 'rooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                 ('male_over_female', 'r4h3', 'r4m3'),
                 ('man12plus_over_women12plus', 'r4h2', 'r4m2'),
                 ('pesioner_over_working', 'hogar_mayor', 'hogar_adul'),
                 ('children_over_working', 'hogar_nin', 'hogar_adul'),
                 ('education_fraction', 'escolari', 'age')
                 #('', '', ''),
                ]
    
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('non_bedrooms', 'rooms', 'bedrooms'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
    
    # aggregation rules over household
    aggs_num = {'age': ['min', 'max', 'mean', 'count'],
                'escolari': ['min', 'max', 'mean', 'std'],
                'fe_education_fraction': ['min', 'max', 'mean', 'std']
               }
    aggs_cat = {'dis': ['mean']}
    for s_ in ['estadocivil', 'parentesco', 'instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean']
    # aggregation over household
    for name_, df_ in [('18', df.query('age >= 18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg
    # do something advanced above...
    
    # Drop SQB variables, as they are just squres of other vars 
    df.drop([f_ for f_ in df.columns if f_.startswith('SQB') or f_ == 'agesq'], axis=1, inplace=True)
    # Drop id's
    df.drop(['Id'], axis=1, inplace=True)
    # Drop repeated columns
    df.drop(['hhsize', 'female', 'area2'], axis=1, inplace=True)
    return df

In [11]:
def encode_data(df):
    '''
    The function does not return, but transforms the input pd.DataFrame
    
    Encodes the Costa Rican Household Poverty Level data 
    following https://www.kaggle.com/c/costa-rican-household-poverty-prediction/discussion/61403#359631
    
    '''
    
    yes_no_map = {'no': 0, 'yes': 1}
    
    df['dependency'] = df['dependency'].replace(yes_no_map).astype(np.float32)
    
    df['edjefe'] = df['edjefe'].replace(yes_no_map).astype(np.float32)
    df['edjefa'] = df['edjefa'].replace(yes_no_map).astype(np.float32)
    
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])

In [13]:
def convert_OHE2LE(df):
    tmp_df = df.copy(deep=True)
    for s_ in ['pared', 'piso', 'techo', 'abastagua', 'sanitario', 'energcocinar', 'elimbasu', 
               'epared', 'etecho', 'eviv', 'estadocivil', 'parentesco', 
               'instlevel', 'lugar', 'tipovivi',
               'manual_elec']:
        if 'manual_' not in s_:
            cols_s_ = [f_ for f_ in df.columns if f_.startswith(s_)]
        elif 'elec' in s_:
            cols_s_ = ['public', 'planpri', 'noelec', 'coopele']
        sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
        #deal with those OHE, where there is a sum over columns == 0
        if 0 in sum_ohe:
            print('The OHE in {} is incomplete. A new column will be added before label encoding'
                  .format(s_))
            # dummy colmn name to be added
            col_dummy = s_+'_dummy'
            # add the column to the dataframe
            tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis=1) == 0).astype(np.int8)
            # add the name to the list of columns to be label-encoded
            cols_s_.append(col_dummy)
            # proof-check, that now the category is complete
            sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
            if 0 in sum_ohe:
                 print("The category completion did not work")
        tmp_cat = tmp_df[cols_s_].idxmax(axis=1)
        tmp_df[s_ + '_LE'] = LabelEncoder().fit_transform(tmp_cat).astype(np.int16)
        if 'parentesco1' in cols_s_:
            cols_s_.remove('parentesco1')
        tmp_df.drop(cols_s_, axis=1, inplace=True)
    return tmp_df

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9557 entries, 0 to 9556
Columns: 143 entries, Id to Target
dtypes: float64(8), int64(130), object(5)
memory usage: 10.4+ MB


In [22]:
def process_df(df_):
    # fix categorical features
    encode_data(df_)
    #fill in missing values based on https://www.kaggle.com/mlisovyi/missing-values-in-the-data
    for f_ in ['v2a1', 'v18q1', 'meaneduc', 'SQBmeaned']:
        df_[f_] = df_[f_].fillna(0)
    df_['rez_esc'] = df_['rez_esc'].fillna(-1)
    # do feature engineering and drop useless columns
    return do_features(df_)

train = process_df(train)
test = process_df(test)

In [23]:
train.head()

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,agg18_parentesco12_MEAN,agg18_instlevel1_MEAN,agg18_instlevel2_MEAN,agg18_instlevel3_MEAN,agg18_instlevel4_MEAN,agg18_instlevel5_MEAN,agg18_instlevel6_MEAN,agg18_instlevel7_MEAN,agg18_instlevel8_MEAN,agg18_instlevel9_MEAN
0,190000.0,0,3,0,1,1,0,0.0,0,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,135000.0,0,4,0,1,1,1,1.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0,8,0,1,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,180000.0,0,5,0,1,1,1,1.0,0,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,180000.0,0,5,0,1,1,1,1.0,0,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [25]:
list(train)

['v2a1',
 'hacdor',
 'rooms',
 'hacapo',
 'v14a',
 'refrig',
 'v18q',
 'v18q1',
 'r4h1',
 'r4h2',
 'r4h3',
 'r4m1',
 'r4m2',
 'r4m3',
 'r4t1',
 'r4t2',
 'r4t3',
 'tamhog',
 'tamviv',
 'escolari',
 'rez_esc',
 'paredblolad',
 'paredzocalo',
 'paredpreb',
 'pareddes',
 'paredmad',
 'paredzinc',
 'paredfibras',
 'paredother',
 'pisomoscer',
 'pisocemento',
 'pisoother',
 'pisonatur',
 'pisonotiene',
 'pisomadera',
 'techozinc',
 'techoentrepiso',
 'techocane',
 'techootro',
 'cielorazo',
 'abastaguadentro',
 'abastaguafuera',
 'abastaguano',
 'public',
 'planpri',
 'noelec',
 'coopele',
 'sanitario1',
 'sanitario2',
 'sanitario3',
 'sanitario5',
 'sanitario6',
 'energcocinar1',
 'energcocinar2',
 'energcocinar3',
 'energcocinar4',
 'elimbasu1',
 'elimbasu2',
 'elimbasu3',
 'elimbasu4',
 'elimbasu5',
 'elimbasu6',
 'epared1',
 'epared2',
 'epared3',
 'etecho1',
 'etecho2',
 'etecho3',
 'eviv1',
 'eviv2',
 'eviv3',
 'dis',
 'male',
 'estadocivil1',
 'estadocivil2',
 'estadocivil3',
 'estado

In [28]:
def train_test_apply_func(train_, test_, func_):
    test_['Target'] = 0
    xx = pd.concat([train_, test_])

    xx_func = func_(xx)
    train_ = xx_func.iloc[:train_.shape[0], :]
    test_  = xx_func.iloc[train_.shape[0]:, :].drop('Target', axis=1)

    del xx, xx_func
    return train_, test_

In [29]:
train, test = train_test_apply_func(train, test, convert_OHE2LE)

The OHE in techo is incomplete. A new column will be added before label encoding
The OHE in instlevel is incomplete. A new column will be added before label encoding
The OHE in manual_elec is incomplete. A new column will be added before label encoding


In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9557 entries, 0 to 9556
Columns: 120 entries, Target to manual_elec_LE
dtypes: float32(64), float64(5), int16(16), int64(35)
memory usage: 5.6 MB


In [31]:
cols_2_ohe = ['eviv_LE', 'etecho_LE', 'epared_LE', 'elimbasu_LE', 
              'energcocinar_LE', 'sanitario_LE', 'manual_elec_LE',
              'pared_LE']
cols_nums = ['age', 'meaneduc', 'dependency', 
             'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total',
             'bedrooms', 'overcrowding']

def convert_geo2aggs(df_):
    tmp_df = pd.concat([df_[(['lugar_LE', 'idhogar']+cols_nums)],
                        pd.get_dummies(df_[cols_2_ohe], 
                                       columns=cols_2_ohe)],axis=1)
    geo_agg = tmp_df.groupby(['lugar_LE','idhogar']).mean().groupby('lugar_LE').mean().astype(np.float32)
    geo_agg.columns = pd.Index(['geo_' + e + '_MEAN' for e in geo_agg.columns.tolist()])
    
    del tmp_df
    return df_.join(geo_agg, how='left', on='lugar_LE')

train, test = train_test_apply_func(train, test, convert_geo2aggs)